In [1]:
!pip install safetensors transformers sentencepiece protobuf pandas datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 259.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 169.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 402.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.4/563.4 kB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 295.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 799.0/799.0 kB 157.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 287.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 270.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 110.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import pandas as pd

df = pd.read_csv('monolingual_dataset/discussion_news_2019_from_1mil_to_6mil_cleaned.csv')
df = df.drop_duplicates()
df = df.dropna()
df = df.reset_index()
df = df.drop(columns='index')
def capitalize_first_letter(text):
    return text.capitalize()

df['english'] = df['english'].apply(capitalize_first_letter)
df=df.iloc[3000000:5000000]
df

,english
3000000,With macron and merkel now taking over the eur...
3000001,Its a terrifying thought the charming snake in...
3000002,Just have a general election - then the 402 co...
3000003,Unless one party comes out for remain and the ...
3000004,You can always vote for an independent leave c...
...,...
4302712,"If the terrain requires it, then even more so ..."
4302713,The difference is - with a gas or hybrid you c...
4302714,With the rivian specs and current battery tech...
4302715,Looks like a great product!


In [3]:
from transformers import MarianTokenizer, MarianMTModel

model_name = "Helsinki-NLP/opus-mt-en-fr"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [4]:
from datasets import Dataset
import torch

# Define the translation function
def translate_batch_to_french(batch, model, tokenizer):
    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Tokenize the batch of English input texts
    encoded_en = tokenizer(batch['english'], return_tensors="pt", padding=True, truncation=True).to(device)  # Move to GPU

    # Generate the translation from English to French
    with torch.no_grad():
        generated_tokens = model.generate(**encoded_en)

    # Decode the generated tokens to obtain the French translations
    french_translations = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    # Add the French translations to the batch
    batch['french'] = french_translations
    return batch

# Create a Dataset object from your DataFrame
dataset = Dataset.from_pandas(df[['english']])  # assuming df is a pandas DataFrame

# Apply the translation function to each batch of the dataset
batch_size = 128
translated_dataset = dataset.map(
    lambda batch: translate_batch_to_french(batch, model, tokenizer), 
    batched=True, 
    batch_size=batch_size
)

# Display the updated dataset
translated_dataset


model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

Map:   0%|          | 0/1302717 [00:00<?, ? examples/s]

Dataset({
    features: ['english', 'french'],
    num_rows: 1302717
})

In [5]:
file_path = "training_dataset/documentary_news_dataset_4to6_from_back_translation.csv"

translated_dataset.to_csv(file_path, index=False)

Creating CSV from Arrow format:   0%|          | 0/1303 [00:00<?, ?ba/s]

218464436

## below is for time estimation of the back translation

In [17]:
import pandas as pd

df = pd.read_csv('monolingual_dataset/discussion_news_2019_from_1mil_to_6mil_cleaned.csv')
df = df.drop_duplicates()
df = df.dropna()
df = df.reset_index()
df = df.drop(columns='index')
def capitalize_first_letter(text):
    return text.capitalize()

df['english'] = df['english'].apply(capitalize_first_letter)
df=df.iloc[3000000:5000000]
df

,english
3000000,With macron and merkel now taking over the eur...
3000001,Its a terrifying thought the charming snake in...
3000002,Just have a general election - then the 402 co...
3000003,Unless one party comes out for remain and the ...
3000004,You can always vote for an independent leave c...
...,...
4302712,"If the terrain requires it, then even more so ..."
4302713,The difference is - with a gas or hybrid you c...
4302714,With the rivian specs and current battery tech...
4302715,Looks like a great product!


In [ ]:
from datasets import Dataset
import torch

# Define the translation function
def translate_batch_to_french(batch, model, tokenizer):
    # Move model to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    
    # Tokenize the batch of English input texts
    encoded_en = tokenizer(batch['english'], return_tensors="pt", padding=True, truncation=True).to(device)  # Move to GPU

    # Generate the translation from English to French
    with torch.no_grad():
        generated_tokens = model.generate(**encoded_en)

    # Decode the generated tokens to obtain the French translations
    french_translations = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    # Add the French translations to the batch
    batch['french'] = french_translations
    return batch

# Create a Dataset object from your DataFrame
dataset = Dataset.from_pandas(df[['english']])  # assuming df is a pandas DataFrame

# Apply the translation function to each batch of the dataset
batch_size = 128
translated_dataset = dataset.map(
    lambda batch: translate_batch_to_french(batch, model, tokenizer), 
    batched=True, 
    batch_size=batch_size
)

# Display the updated dataset
translated_dataset


Map:   0%|          | 0/1302717 [00:00<?, ? examples/s]

In [ ]:
file_path = "training_dataset/documentary_news_dataset_4to6_from_back_translation.csv"

translated_dataset.to_csv(file_path, index=False)